# Add Covariates to Dubosson Dataset

In [1]:
import pandas as pd
import datetime
import numpy as np

## Load Glucose data

In [2]:
# Import glocose data file
data = pd.read_csv("./raw_data/Dubosson2018_processed.csv")
data

,id,time,gl
0,1,2014-10-01 19:14:00,185.4
1,1,2014-10-01 19:19:00,178.2
2,1,2014-10-01 19:24:00,176.4
3,1,2014-10-01 19:29:00,172.8
4,1,2014-10-01 19:34:00,169.2
...,...,...,...
8050,9,2014-10-03 12:20:19,88.2
8051,9,2014-10-03 12:25:19,75.6
8052,9,2014-10-03 12:30:19,59.4
8053,9,2014-10-03 12:35:19,48.6


## Insulin

In [3]:
# Loop over the folder of each subject and merge files with insulin data by id
subject_ids = ["001", "002", "003", "004", "005", "006", "007", "008", "009"]

df_list = []
for subject_id in subject_ids:
    subject_data = pd.read_csv(f"raw_covariates/dubosson/diabetes_subset_pictures-glucose-food-insulin/{subject_id}/insulin.csv")
    subject_data["id"] = subject_id
    df_list.append(subject_data)

insulin_data = pd.concat(df_list, axis=0, ignore_index=True)
insulin_data

,date,time,fast_insulin,slow_insulin,comment,id
0,2014-10-01,10:06:00,7.0,NaN,NaN,001
1,2014-10-01,16:50:00,4.0,NaN,NaN,001
2,2014-10-01,19:28:00,6.0,NaN,NaN,001
3,2014-10-01,22:27:00,8.0,NaN,NaN,001
4,2014-10-01,23:48:00,0.0,31.0,NaN,001
...,...,...,...,...,...,...
121,2014-10-03,22:00:00,NaN,18.0,NaN,009
122,2014-10-04,06:00:00,3.0,NaN,NaN,009
123,2014-10-04,12:00:00,4.0,NaN,NaN,009
124,2014-10-04,19:00:00,4.0,NaN,NaN,009


In [4]:
# Create one daytime column 
insulin_data['date'] = pd.to_datetime(insulin_data['date'])
insulin_data['time'] = pd.to_datetime(insulin_data['time'], format='%H:%M:%S').dt.time
insulin_data['datetime'] = insulin_data.apply(lambda x: datetime.datetime.combine(x['date'], x['time']), axis=1)

# Drop Date, Time, Comment columns
insulin_data.drop(["date", "time", "comment"], axis=1, inplace=True)

# Replace NaNs with zeroes
insulin_data['fast_insulin'].fillna(0, inplace=True)
insulin_data['slow_insulin'].fillna(0, inplace=True)

# Covert subject ids to int64 to match with "data" ids
insulin_data['id'] = insulin_data['id'].astype(int)

# Change the type of time variable in glucosse dataset
data['time'] = pd.to_datetime(data['time'])

insulin_data

,fast_insulin,slow_insulin,id,datetime
0,7.0,0.0,1,2014-10-01 10:06:00
1,4.0,0.0,1,2014-10-01 16:50:00
2,6.0,0.0,1,2014-10-01 19:28:00
3,8.0,0.0,1,2014-10-01 22:27:00
4,0.0,31.0,1,2014-10-01 23:48:00
...,...,...,...,...
121,0.0,18.0,9,2014-10-03 22:00:00
122,3.0,0.0,9,2014-10-04 06:00:00
123,4.0,0.0,9,2014-10-04 12:00:00
124,4.0,0.0,9,2014-10-04 19:00:00


In [5]:
# Merge the two datasets based on "id"
df = insulin_data.merge(data, on='id')

# For each row in insulin_data, calculate the absolute difference
df['diff'] = (df['datetime'] - df['time']).abs()

# Find the index of the minimum difference for each subject and each insulin date-time
idx = df.groupby(['id', 'datetime'])['diff'].idxmin()

# Use that index to retrieve the corresponding "time" value
df_final = df.loc[idx, ['id', 'datetime', 'time']]
df_final.rename(columns={'id': 'id', 'time': 'closest_time'}, inplace=True)

# Add the closest time as a new column in insulin_data
result = insulin_data.merge(df_final, on=['id', 'datetime'], how='left')

# Calculate the difference between the closest time and datetime in minutes
result.loc[:, 'time_diff'] = np.abs((result['closest_time'] - result['datetime']) / np.timedelta64(1, 'm'))

# Keep only the rows where the absolute difference is less than or equal to 5 minutes
result = result.loc[result['time_diff'] <= 5, :]

# Some rows have exact the same closest_time when a person took fast and slow insulin at the same time. 
# Merge these duplicate rows in one row
result = result.groupby(["id", "closest_time"]).agg({"fast_insulin": "sum", "slow_insulin": "sum"}).reset_index()

In [6]:
# Merge glucose and insulin datasets
data_cov = data.merge(result, how='left', left_on=['id', 'time'], right_on=['id', 'closest_time'])

# Drop closest_time column
data_cov.drop(["closest_time"], axis=1, inplace=True)

# Replace NaN with zerows
data_cov = data_cov.fillna(0)

data_cov

,id,time,gl,fast_insulin,slow_insulin
0,1,2014-10-01 19:14:00,185.4,0.0,0.0
1,1,2014-10-01 19:19:00,178.2,0.0,0.0
2,1,2014-10-01 19:24:00,176.4,0.0,0.0
3,1,2014-10-01 19:29:00,172.8,6.0,0.0
4,1,2014-10-01 19:34:00,169.2,0.0,0.0
...,...,...,...,...,...
8050,9,2014-10-03 12:20:19,88.2,0.0,0.0
8051,9,2014-10-03 12:25:19,75.6,0.0,0.0
8052,9,2014-10-03 12:30:19,59.4,0.0,0.0
8053,9,2014-10-03 12:35:19,48.6,0.0,0.0


## Summary Statistic